# Data extraction via MT5 Api

Each trading symbol has its own process for the wine mt5 api. This is so that in production, we're able to reduce latency (at the cost of higher memory consumption) by pulling from one or more brokers in parallel.

### IMPORTANT - start mt5 apis

Before running this notebooks, first launch the mt5 apis from the terminal using the command: releat launch-all-mt5-apis

Launching wine processes from jupyter notebooks is unstable and mostly fails to start

In [ ]:
from releat.utils.configs.constants import mt5_api_port_map
import requests
from concurrent.futures import ThreadPoolExecutor
from releat.workflows.service_manager import kill_processes, get_pids, stop_mt5
from releat.utils.logging import get_logger
import logging
logger = get_logger(__name__, log_level=logging.INFO)

In [ ]:
broker_symbol_ports = []
for broker, port_map in mt5_api_port_map.items():
    for symbol, port in port_map.items():
        try:
            requests.get(f"http://127.0.0.1:{port}/healthcheck").json()
            logger.info(f"healthcheck {broker} {symbol} on port {port} success")
            if symbol != "general":
                broker_symbol_ports.append([broker,symbol,port])
        except:
            logger.error("Check that mt5 apis have been started - See the first markdown cell of this notebook")

In [ ]:
def get_data(broker_symbol_port):
    """Simple function to download data"""
    _, symbol, port = broker_symbol_port
    
    d_request = {
        "symbol": symbol,
        "dt0": "2023-09-06 12:00:00.000",
        "dt1": "2023-09-06 13:01:00.000",
    }

    data = requests.get(
        f"http://127.0.0.1:{port}/get_tick_data",
        json=d_request,
        timeout=120,
    ).json()
    
    return data

In [ ]:
# show example output of function
get_data(broker_symbol_ports)[:3]

### Compare data extraction speed

When run in parallel, the time taken to download data is at least 2x as fast as compared to in sequence. Note results will vary depending on internet speed and whether results are cached

In [ ]:
%%timeit
for broker_symbol_port in broker_symbol_ports:
    get_data(broker_symbol_port)

In [ ]:
pool = ThreadPoolExecutor(5)

In [ ]:
%%timeit
list(pool.map(get_data,broker_symbol_ports))

### Clean processes by deleting MT5 instances and apis

In [ ]:
# kill mt5
stop_mt5()

In [ ]:
# kill mt5 api process ids
pids = get_pids("wineserver")
kill_processes(pids)
print(f"mt5 apis stopped - process ids {pids} killed")

# kill wine processes
pids = get_pids("python.exe")
kill_processes(pids)